In [1]:
from scipy.io import loadmat, savemat
import numpy as np
import time
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn import cross_validation

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
from keras import backend as K

from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation
from keras.wrappers.scikit_learn import KerasRegressor

from keras.optimizers import SGD, adam, nadam, Adagrad
from keras.regularizers import l1,l2

from keras.callbacks import EarlyStopping, CSVLogger
from keras.losses import mean_squared_logarithmic_error


import os
import os.path
import sys
import argparse
import time
import csv

D:\Users\Vishwesh\Anaconda3\envs\deep_l\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15781502454168053974
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 5072859955
locality {
  bus_id: 1
}
incarnation: 12167383872144736457
physical_device_desc: "device: 0, name: GeForce GTX 970M, pci bus id: 0000:01:00.0"
]


In [4]:
train_input_data_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\train_input_mapmri_r8.mat'
train_output_data_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\train_output_shore.mat'
train_input_data_path = os.path.normpath(train_input_data_path)
train_output_data_path = os.path.normpath(train_output_data_path)

test_input_data_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\test_input_mapmri_r8.mat'
test_output_data_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\test_output_shore.mat'
test_input_data_path = os.path.normpath(test_input_data_path)
test_output_data_path = os.path.normpath(test_output_data_path)

input = loadmat(train_input_data_path)
output = loadmat(train_output_data_path)

#X = np.array(input['train_input_shore'])
X = np.array(input['train_input_mapmri_r8'])
y = np.array(output['train_output_shore'])

# Get dimensions of arrays
print('Training Data Information \n')
x_size = X.shape
print('Input Array Shape',x_size)
y_size = y.shape
print ('Output Array Shape',y_size)

test_input = loadmat(test_input_data_path)
test_output = loadmat(test_output_data_path)

#X_test = np.array(test_input['test_input_shore'])
X_test = np.array(test_input['test_input_mapmri_r8'])
y_test = np.array(test_output['test_output_shore'])

# Get dimensions of Test data
print('Testing Data information \n')
x_size = X_test.shape
print('Test Input Shape',x_size)
y_size = y_test.shape
print('Test Output Shape',y_size)

Training Data Information 

Input Array Shape (49995, 95)
Output Array Shape (49995, 66)
Testing Data information 

Test Input Shape (7272, 95)
Test Output Shape (7272, 66)


In [7]:
# Build Generator Code
def build_generator_A(input_B,name='genA'):
    
    
    y = Dense(400, activation='relu')(input_B)
    y = Dense(66, activation='relu')(y)
    y = Dense(200, activation='relu')(y)
    y = Dense(95, activation='linear')(y)
    return y

    #model = Sequential()
    # Input layer with dimension 1 and hidden layer i with 128 neurons.
    #model.add(Dense(95, input_shape=(95,)))
    #model.add(Dense(400))
    #model.add(Activation("relu"))
    # Hidden layer j with 64 neurons plus activation layer.
    #model.add(Dense(66))
    #model.add(Activation("relu"))
    # Hidden layer k with 64 neurons.
    #model.add(Dense(200))
    #model.add(Activation("relu"))
    #model.add(Dense(200))
    # Output Layer.
    #model.add(Dense(66))
 
    # Model is derived and compiled using mean square error as loss
    # function, accuracy as metric and gradient descent optimizer.
    #model.compile(loss=mean_squared_logarithmic_error, optimizer='nadam', metrics=['mse','mae'])
    #model.summary()
    

def build_generator_B(input_A,name='genB'):
    
    #model = Sequential()
    # Input layer with dimension 1 and hidden layer i with 128 neurons.
    x = Dense(400, activation='relu')(input_A)
    x = Dense(66, activation='relu')(x)
    x = Dense(200, activation='relu')(x)
    x = Dense(66, activation='linear')(x)
    return x

    #model.add(Dense(66, input_shape=(66,)))
    #model.add(Dense(400))
    #model.add(Activation("relu"))
    # Hidden layer j with 64 neurons plus activation layer.
    #model.add(Dense(66))
    #model.add(Activation("relu"))
    # Hidden layer k with 64 neurons.
    #model.add(Dense(200))
    #model.add(Activation("relu"))
    #model.add(Dense(200))
    # Output Layer.
    #model.add(Dense(95))
    
    
def build_dis_A(input_B,name='disA'):
    
    x = Dense(400, activation='relu')(input_B)
    x = Dense(66, activation='relu')(x)
    x = Dense(200, activation='relu')(x)
    x = Dense(66, activation='linear')(x)
    x = Dense(1, activation='sigmoid')(x)
    return x
    #model = Sequential()
    # Input layer with dimension 1 and hidden layer i with 128 neurons.
    #model.add(Dense(66, input_shape=(66,)))
    #model.add(Dense(400))
    #model.add(Activation("relu"))
    # Hidden layer j with 64 neurons plus activation layer.
    #model.add(Dense(200))
    #model.add(Activation("relu"))
    # Hidden layer k with 64 neurons.
    #model.add(Dense(66))
    #model.add(Activation("relu"))
    #model.add(Dense(200))
    #model.add(Activation("relu"))
    #model.add(Dense(1,activation="sigmoid"))
    #return model
    
def build_dis_B(input_A,name='disB'):
    
    x = Dense(400, activation='relu')(input_A)
    x = Dense(66, activation='relu')(x)
    x = Dense(200, activation='relu')(x)
    x = Dense(66, activation='linear')(x)
    x = Dense(1, activation='sigmoid')(x)
    return x
    
    #model = Sequential()
    # Input layer with dimension 1 and hidden layer i with 128 neurons.
    #model.add(Dense(95, input_shape=(95,)))
    #model.add(Dense(400))
    #model.add(Activation("relu"))
    # Hidden layer j with 64 neurons plus activation layer.
    #model.add(Dense(200))
    #model.add(Activation("relu"))
    # Hidden layer k with 64 neurons.
    #model.add(Dense(66))
    #model.add(Activation("relu"))
    #model.add(Dense(200))
    #model.add(Activation("relu"))
    #model.add(Dense(1,activation="sigmoid"))
    #return model

In [9]:
batch_size = 100
input_A = tf.placeholder(tf.float32, [batch_size, 95, 1], name="input_A")
input_B = tf.placeholder(tf.float32, [batch_size, 66, 1], name="input_B")

gen_B = build_generator_B(input_A, name="generator_AtoB")
gen_A = build_generator_A(input_B, name="generator_BtoA")
dec_A = build_dis_A(input_A, name="discriminator_A")
dec_B = build_dis_B(input_B, name="discriminator_B")

dec_gen_A = build_dis_B(gen_A, "discriminator_A")
dec_gen_B = build_dis_A(gen_B, "discriminator_B")
cyc_A = build_generator_B(gen_B, "generator_BtoA")
cyc_B = build_generator_A(gen_A, "generator_AtoB")


In [10]:
D_A_loss_1 = tf.reduce_mean(tf.squared_difference(dec_A,1))
D_B_loss_1 = tf.reduce_mean(tf.squared_difference(dec_B,1))

D_A_loss_2 = tf.reduce_mean(tf.square(dec_gen_A))
D_B_loss_2 = tf.reduce_mean(tf.square(dec_gen_B))

D_A_loss = (D_A_loss_1 + D_A_loss_2)/2
D_B_loss = (D_B_loss_1 + D_B_loss_2)/2

g_loss_B_1 = tf.reduce_mean(tf.squared_difference(dec_gen_A,1))
g_loss_A_1 = tf.reduce_mean(tf.squared_difference(dec_gen_A,1))

cyc_loss = tf.reduce_mean(tf.abs(input_A-cyc_A)) + tf.reduce_mean(tf.abs(input_B-cyc_B))

g_loss_A = g_loss_A_1 + 10*cyc_loss
g_loss_B = g_loss_B_1 + 10*cyc_loss

In [12]:
optimizer = tf.train.AdamOptimizer(0.0001)
d_A_trainer = optimizer.minimize(d_loss_A, var_list=d_A_vars)
d_B_trainer = optimizer.minimize(d_loss_B, var_list=d_B_vars)
g_A_trainer = optimizer.minimize(g_loss_A, var_list=g_A_vars)
g_B_trainer = optimizer.minimize(g_loss_B, var_list=g_B_vars)

NameError: name 'd_loss_A' is not defined